 # Install/Import Packages and Download/Unzip Dataset

 ---

 The following code installs d3rlpy, imports all necessary dependencies, and downloads/unzips the dataset from Zenodo.

In [ ]:
!pip install d3rlpy==2.6.1 gymnasium==0.29.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827621 sha256=a68e4a281a606827d1b4571789716a5ec899eb64abce7fc8828a907eebb1c455
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not curr

In [ ]:
from google.colab import files
import numpy as np
import d3rlpy
import os

In [ ]:
!wget -O full_dataset.zip "https://zenodo.org/record/13830810/files/full_dataset.zip?download=1"

--2024-09-25 20:18:24--  https://zenodo.org/record/13830810/files/full_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/13830810/files/full_dataset.zip [following]
--2024-09-25 20:18:25--  https://zenodo.org/records/13830810/files/full_dataset.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 2161582746 (2.0G) [application/octet-stream]
Saving to: ‘full_dataset.zip’

full_dataset.zip    100%[===================>]   2.01G  18.8MB/s    in 90s     

2024-09-25 20:19:55 (22.8 MB/s) - ‘full_dataset.zip’ saved [2161582746/2161582746]



In [ ]:
!unzip full_dataset.zip -d /content/
data_dir = '/content/content/data/data'

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/data/data/ep_38406.npz  
  inflating: /content/content/data/data/ep_45886.npz  
  inflating: /content/content/data/data/ep_80720.npz  
  inflating: /content/content/data/data/ep_41811.npz  
  inflating: /content/content/data/data/ep_74361.npz  
  inflating: /content/content/data/data/ep_36142.npz  
  inflating: /content/content/data/data/ep_92647.npz  
  inflating: /content/content/data/data/ep_35769.npz  
  inflating: /content/content/data/data/ep_70452.npz  
  inflating: /content/content/data/data/ep_89001.npz  
  inflating: /content/content/data/data/ep_92324.npz  
  inflating: /content/content/data/data/ep_21612.npz  
  inflating: /content/content/data/data/ep_51788.npz  
  inflating: /content/content/data/data/ep_51083.npz  
  inflating: /content/content/data/data/ep_54442.npz  
  inflating: /content/content/data/data/ep_97628.npz  
  inflating: /content/content/data/data/ep_26720.npz  
  inflating: /

# Prepare Dataset for d3rlpy

---

The following code loads all of the data into arrays and creates a d3rlpy dataset.

In [ ]:
total_steps = 5000000

all_observations = np.empty((total_steps, 3, 64, 64), dtype=np.uint8)
all_actions = np.empty((total_steps, 3), dtype=np.float32)
all_rewards = np.empty((total_steps,), dtype=np.float32)
all_terminals = np.empty((total_steps,), dtype=bool)

current_index = 0

for i in range(1, 99523):
    episode_file = os.path.join(data_dir, f'ep_{i}.npz')
    episode_data = np.load(episode_file)

    num_steps = episode_data['terminals'].shape[0]

    all_observations[current_index:current_index + num_steps] = np.transpose(episode_data['observations'], (0, 3, 1, 2))
    all_actions[current_index:current_index + num_steps] = episode_data['actions']
    all_rewards[current_index:current_index + num_steps] = episode_data['rewards']
    all_terminals[current_index:current_index + num_steps] = episode_data['terminals']

    current_index += num_steps

all_observations = all_observations[:current_index]
all_actions = all_actions[:current_index]
all_rewards = all_rewards[:current_index]
all_terminals = all_terminals[:current_index]

In [ ]:
dataset = d3rlpy.dataset.MDPDataset(
    observations=all_observations,
    actions=all_actions,
    rewards=all_rewards,
    terminals=all_terminals,
)

2024-09-25 20:34.23 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2024-09-25 20:34.23 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2024-09-25 20:34.23 [info     ] Action size has been automatically determined. action_size=3


# Model Training

---

The following code trains a behavior cloning algorithm for 50 epochs at 10000 steps per epoch.

In [ ]:
BC = d3rlpy.algos.BCConfig(observation_scaler=d3rlpy.preprocessing.PixelObservationScaler(),).create(device='cuda')
BC.fit(dataset=dataset, n_steps=500000, n_steps_per_epoch=10000)

2024-09-25 20:34.27 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 64, 64)]), action_signature=Signature(dtype=[dtype('float32')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2024-09-25 20:34.27 [info     ] Directory is created at d3rlpy_logs/BC_20240925203427
2024-09-25 20:34.27 [debug    ] Building models...            
2024-09-25 20:34.29 [debug    ] Models have been built.       
2024-09-25 20:34.29 [info     ] Parameters                     params={'observation_shape': [3, 64, 64], 'action_size': 3, 'config': {'type': 'bc', 'params': {'batch_size': 100, 'gamma': 0.99, 'observation_scaler': {'type': 'pixel', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 0.001, 'policy_type': 'deterministic', 'optim_factory': {'type': '

Epoch 1/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 20:36.19 [info     ] BC_20240925203427: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.005085845327377319, 'time_algorithm_update': 0.005729454493522644, 'loss': 0.016795805045915768, 'time_step': 0.010919874453544616} step=10000
2024-09-25 20:36.19 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203427/model_10000.d3


Epoch 2/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 20:37.50 [info     ] BC_20240925203427: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.003478450655937195, 'time_algorithm_update': 0.0054514554977416994, 'loss': 0.009716768994787708, 'time_step': 0.0090264888048172} step=20000
2024-09-25 20:37.50 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203427/model_20000.d3


...

Epoch 49/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:50.17 [info     ] BC_20240925203427: epoch=49 step=490000 epoch=49 metrics={'time_sample_batch': 0.0035537765979766845, 'time_algorithm_update': 0.005517019534111023, 'loss': 0.006368977200239897, 'time_step': 0.009173365831375123} step=490000
2024-09-25 21:50.17 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203427/model_490000.d3


Epoch 50/50:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-09-25 21:51.50 [info     ] BC_20240925203427: epoch=50 step=500000 epoch=50 metrics={'time_sample_batch': 0.0035849644899368285, 'time_algorithm_update': 0.005532838296890259, 'loss': 0.006381155525543727, 'time_step': 0.00922063581943512} step=500000
2024-09-25 21:51.50 [info     ] Model parameters are saved to d3rlpy_logs/BC_20240925203427/model_500000.d3


[(1,
  {'time_sample_batch': 0.005085845327377319,
   'time_algorithm_update': 0.005729454493522644,
   'loss': 0.016795805045915768,
   'time_step': 0.010919874453544616}),
 (2,
  {'time_sample_batch': 0.003478450655937195,
   'time_algorithm_update': 0.0054514554977416994,
   'loss': 0.009716768994787708,
   'time_step': 0.0090264888048172}),
 (3,
  {'time_sample_batch': 0.0035152724742889404,
   'time_algorithm_update': 0.005485492253303528,
   'loss': 0.008580144820408895,
   'time_step': 0.009100013375282288}),
 (4,
  {'time_sample_batch': 0.003388635230064392,
   'time_algorithm_update': 0.005326024293899536,
   'loss': 0.008064004987711086,
   'time_step': 0.00881223132610321}),
 (5,
  {'time_sample_batch': 0.0033934303283691407,
   'time_algorithm_update': 0.005344832801818847,
   'loss': 0.007780826750583947,
   'time_step': 0.008835095381736756}),
 (6,
  {'time_sample_batch': 0.0035825555562973023,
   'time_algorithm_update': 0.0055952815532684325,
   'loss': 0.00757883151734

In [ ]:
!zip -r model_full_dataset.zip d3rlpy_logs

  adding: d3rlpy_logs/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925203427/ (stored 0%)
  adding: d3rlpy_logs/BC_20240925203427/model_90000.d3 (deflated 44%)
  adding: d3rlpy_logs/BC_20240925203427/time_algorithm_update.csv (deflated 60%)
  adding: d3rlpy_logs/BC_20240925203427/time_step.csv (deflated 60%)
  adding: d3rlpy_logs/BC_20240925203427/model_30000.d3 (deflated 27%)
  adding: d3rlpy_logs/BC_20240925203427/model_320000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_440000.d3 (deflated 48%)
  adding: d3rlpy_logs/BC_20240925203427/model_340000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_250000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_160000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_290000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_130000.d3 (deflated 46%)
  adding: d3rlpy_logs/BC_20240925203427/model_500000.d3 (deflated 47%)
  adding: d3rlpy_logs/BC_20240925203427/model_140000.d